This notebook loops through the local experiment folder and removes the large files/variables associated with each experiment and stores a single copy of them in a separate file

In particular, the backtest results pkls have a lot of extra information that is not required

In [15]:
import pandas as pd
from services.experiments import *
import matplotlib.pyplot as plt

In [106]:
universe = 'SP'

CardMVO_path = "Experiments//"+universe+"//"+"CardMVO"
CardMVO_experiments = pd.read_pickle(CardMVO_path+"//data_dictionary.pkl")

# add filters here


assert (CardMVO_experiments.groupby(['cardinality_ratio', 'turnover_limit']).uid.count() == 1).all()

unique_card_turnover_df = CardMVO_experiments.groupby(['cardinality_ratio', 'turnover_limit']).uid.first() # unique experiments

#def read_results(CardMVO_experiments):
estimators = CardMVO_experiments.groupby(['estimator', 'EstNumObs', 'k', 'alpha']).first()
estimator_idx = 0

contexts_by_estimator = {}
mu_by_estimator = {}
cov_by_estimator = {}

for uid in unique_card_turnover_df:

    results = import_dict(CardMVO_path, uid, 'backtest_results')

    contexts_by_period = {} #indexed by t; period context, and cov
    mu_by_period = {}
    cov_by_period = {}

    if uid == estimators.iloc[estimator_idx].uid: #on the first uid save stuff
        estimator_key = estimators.index[estimator_idx]
        for t in results.keys():
            period_results = results[t]
            if 'optimization_params' in period_results.keys():
                contexts_by_period[t] = period_results['optimization_params']['period_Context']

            if 'mu' in period_results.keys():
                mu_by_period[t] = period_results['mu']

            if 'cov' in period_results.keys():
                cov_by_period[t] = period_results['cov']

        contexts_by_estimator[estimator_key] = contexts_by_period
        mu_by_estimator[estimator_key] = mu_by_period
        cov_by_estimator[estimator_key] = cov_by_period

        estimator_idx = min(len(estimators)-1, estimator_idx+1) #move on to the next estimator

    for t in results.keys():
        period_results = results[t]
        if 'optimization_params' in period_results.keys():
            del results[t]['optimization_params']['period_Context']
            del results[t]['optimization_params']['mu']
            del results[t]['optimization_params']['Q']

        if 'mu' in period_results.keys():
            del results[t]['mu']
        if 'cov' in period_results.keys():
            del results[t]['cov']

            # exiting loop over all experiments
        export_dict(CardMVO_path, uid, results, 'backtest_results')

# exiting loop over all experiments
export_dict(CardMVO_path, '', contexts_by_estimator, 'contexts_by_estimator')
export_dict(CardMVO_path, '', mu_by_estimator, 'mu_by_estimator')
export_dict(CardMVO_path, '', cov_by_estimator, 'cov_by_estimator')

In [107]:
contexts_by_estimator.keys()

dict_keys([('exponential_weighted_estimator_shrinkage', '750', '120', '0.01')])

In [108]:
results[0].keys()

dict_keys(['obj_value', 'time', 'optimality gap', 'x', 'z', 'feasible_solution', 'optimization_params', 'calEnd'])

In [109]:
results[0]['optimization_params']

{'targetRet': 0.4044358812724712,
 'previous_portfolio': None,
 'K': 286,
 'k': 120,
 'alpha': 0.010000000000000009,
 'premium': 2,
 'EstNumObs': 750,
 'ticker_index': 0,
 'target_return_strategy': <function services.environment_functions.premium_target(mu, premium)>,
 'target_return_strategy_args': ['premium'],
 'turnover_limit': 1.25,
 'turnover_constraints': True,
 'cardinality_ratio': 1,
 'MipGap': 0.05,
 'limit_time': 20,
 'bigMStrategy': <function services.big_m_strategies.HyperparameterBigMStrategy(**kwargs)>,
 'bigM_limit_time': 20,
 'bigM_MipGap': 0.05,
 'bigM_SolutionLimit': 10,
 'SkipSOCP': True,
 'tightening_iter_lim': 1,
 'SVMMVO_bigM_time_limit': 5,
 'SVMMVO_MipGap': 0.1,
 'SVMMVO_SolutionLimit': 5,
 'SOCP_limit_time': 10,
 'q': 1,
 'C': 0.1,
 'epsilon': 0.1,
 'separable': False,
 'LogToConsole': False,
 'Verbose': False,
 'hyperparams': {'C': [0.1, 1.0, 10.0, 100.0],
  'epsilon': [0.1, 1.0, 10.0, 100.0]},
 'q_alpha': 0.6838,
 'kappa': 0.025}

In [112]:

SVMMVO_path = "Experiments//"+universe+"//"+"SVMMVO"
qs = [1, 2, 3, 5, 10]


for q in qs:
    SVMMVO_path_q = SVMMVO_path+"//"+str(q)
    SVMMVO_experiments_q = pd.read_pickle(SVMMVO_path_q+"//data_dictionary.pkl")

    estimators = SVMMVO_experiments_q.groupby(['estimator', 'EstNumObs', 'k', 'alpha']).first()
    estimator_idx = 0

    contexts_by_estimator = {}
    mu_by_estimator = {}
    cov_by_estimator = {}

    print("q =", q, " runs available: ", SVMMVO_experiments_q.run.unique())

    assert len(SVMMVO_experiments_q.uid) == len(SVMMVO_experiments_q.uid.unique())

    for uid in SVMMVO_experiments_q.uid:
        results = import_dict(SVMMVO_path_q, uid, 'backtest_results')

        contexts_by_period = {} #indexed by t; period context, and cov
        mu_by_period = {}
        cov_by_period = {}

        # new estimator encountered
        # store
        if uid == estimators.iloc[estimator_idx].uid: #on the first uid save stuff
            estimator_key = estimators.index[estimator_idx]
            for t in results.keys():
                period_results = results[t]
                if 'optimization_params' in period_results.keys():
                    contexts_by_period[t] = period_results['optimization_params']['period_Context']

                if 'mu' in period_results.keys():
                    mu_by_period[t] = period_results['mu']

                if 'cov' in period_results.keys():
                    cov_by_period[t] = period_results['cov']

            contexts_by_estimator[estimator_key] = contexts_by_period
            mu_by_estimator[estimator_key] = mu_by_period
            cov_by_estimator[estimator_key] = cov_by_period

            estimator_idx = min(len(estimators)-1, estimator_idx+1) #move on to the next estimator

        for t in results.keys():
            period_results = results[t]
            if 'optimization_params' in period_results.keys():
                del results[t]['optimization_params']['period_Context']
                del results[t]['optimization_params']['mu']
                del results[t]['optimization_params']['Q']

            if 'mu' in period_results.keys():
                del results[t]['mu']
            if 'cov' in period_results.keys():
                del results[t]['cov']

            # exiting loop over all experiments
        export_dict(SVMMVO_path_q, uid, results, 'backtest_results')

    # exiting loop over all experiments
    export_dict(SVMMVO_path_q, '', contexts_by_estimator, 'contexts_by_estimator')
    export_dict(SVMMVO_path_q, '', mu_by_estimator, 'mu_by_estimator')
    export_dict(SVMMVO_path_q, '', cov_by_estimator, 'cov_by_estimator')

    # delete the unnesssary files

q = 1  runs available:  ['SunJan212024']
q = 2  runs available:  ['SunJan212024']
q = 3  runs available:  ['SunJan212024']
q = 5  runs available:  ['SunJan212024']
q = 10  runs available:  ['SunJan212024']


In [110]:
results[0].keys()

dict_keys(['obj_value', 'time', 'optimality gap', 'x', 'z', 'feasible_solution', 'optimization_params', 'calEnd'])

In [111]:
results[0]['optimization_params'].keys()

dict_keys(['targetRet', 'previous_portfolio', 'K', 'k', 'alpha', 'premium', 'EstNumObs', 'ticker_index', 'target_return_strategy', 'target_return_strategy_args', 'turnover_limit', 'turnover_constraints', 'cardinality_ratio', 'MipGap', 'limit_time', 'bigMStrategy', 'bigM_limit_time', 'bigM_MipGap', 'bigM_SolutionLimit', 'SkipSOCP', 'tightening_iter_lim', 'SVMMVO_bigM_time_limit', 'SVMMVO_MipGap', 'SVMMVO_SolutionLimit', 'SOCP_limit_time', 'q', 'C', 'epsilon', 'separable', 'LogToConsole', 'Verbose', 'hyperparams', 'q_alpha', 'kappa'])

In [42]:
results[1]['C']

1.0

In [32]:
MVO_results[0].keys()

KeyError: 0

In [37]:
MVO_results['1'][0].keys()

dict_keys(['obj_value', 'time', 'optimality gap', 'x', 'z', 'feasible_solution', 'optimization_params', 'calEnd', 'mu', 'cov'])